In [3]:
# Remove bias & relu / avgpool

In [3]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [13]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import os
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import accuracy_score
mpl.rcParams['figure.dpi'] = 600
from PIL import Image

import sys
sys.path.append("../../../src/")
sys.path.append("../..")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.function import recreate_image                                                                                                                                       

## Hyperparameter

In [5]:
resume = "037-0"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
generated_dir = "/home/lincolnzjx/Desktop/saved/generated/"
root = "/media/lincolnzjx/HardDisk/Datasets/"
backbone = "vgg16"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load model parameter

In [29]:
# model
net = model.Network(backbone=backbone, num_classes=1000)
net.to(device)

# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt) 

> Use original fc
Resume from model from exp: 037 at epoch 0


<All keys matched successfully>

###  Strueture A
ReLU + MaxPool + No Bias

In [22]:
# set bias to zero
for name, param in net.named_parameters():
    if "bias" in name:
        nn.init.zeros_(param)
        param.requires_grad = False
        print(name, " is set to zero and require_grad is false.")

# set bias to non training.
for name, param in net.named_parameters():
    if param.requires_grad is True:
        print(name)
        
# print bias values. 
for name, param in net.named_parameters():
    if "bias" in name:
        print(param)

model.features.0.bias  is set to zero and require_grad is false.
model.features.2.bias  is set to zero and require_grad is false.
model.features.5.bias  is set to zero and require_grad is false.
model.features.7.bias  is set to zero and require_grad is false.
model.features.10.bias  is set to zero and require_grad is false.
model.features.12.bias  is set to zero and require_grad is false.
model.features.14.bias  is set to zero and require_grad is false.
model.features.17.bias  is set to zero and require_grad is false.
model.features.19.bias  is set to zero and require_grad is false.
model.features.21.bias  is set to zero and require_grad is false.
model.features.24.bias  is set to zero and require_grad is false.
model.features.26.bias  is set to zero and require_grad is false.
model.features.28.bias  is set to zero and require_grad is false.
model.fc.0.bias  is set to zero and require_grad is false.
model.fc.3.bias  is set to zero and require_grad is false.
model.fc.6.bias  is set to z

In [25]:
save_path = "/home/lincolnzjx/Desktop/Interpretation/saved/models/050"
!mkdir /home/lincolnzjx/Desktop/Interpretation/saved/models/050
torch.save(net.state_dict(), os.path.join(save_path, "0"))

In [28]:
net.load_state_dict(torch.load(os.path.join(save_path, "0")))
# print bias values. 
for name, param in net.named_parameters():
    if "bias" in name:
        print(param)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

###  Strueture B
ReLU + AvgPool + No Bias

In [46]:
# set bias to zero
for name, param in net.named_parameters():
    if "bias" in name:
        nn.init.zeros_(param)
        param.requires_grad = False
        print(name, " is set to zero and require_grad is false.")

# set bias to non training.
for name, param in net.named_parameters():
    if param.requires_grad is True:
        print(name)
        

        
# print bias values. 
for name, param in net.named_parameters():
    if "bias" in name:
        print(param)

model.features.0.bias  is set to zero and require_grad is false.
model.features.2.bias  is set to zero and require_grad is false.
model.features.5.bias  is set to zero and require_grad is false.
model.features.7.bias  is set to zero and require_grad is false.
model.features.10.bias  is set to zero and require_grad is false.
model.features.12.bias  is set to zero and require_grad is false.
model.features.14.bias  is set to zero and require_grad is false.
model.features.17.bias  is set to zero and require_grad is false.
model.features.19.bias  is set to zero and require_grad is false.
model.features.21.bias  is set to zero and require_grad is false.
model.features.24.bias  is set to zero and require_grad is false.
model.features.26.bias  is set to zero and require_grad is false.
model.features.28.bias  is set to zero and require_grad is false.
model.fc.0.bias  is set to zero and require_grad is false.
model.fc.3.bias  is set to zero and require_grad is false.
model.fc.6.bias  is set to z

In [47]:
for index in net.model.features._modules:
    layer = net.model.features._modules[index] 
    if isinstance(layer, nn.MaxPool2d):
        net.model.features._modules[index] = nn.AvgPool2d(kernel_size=2,
                                                          stride=2,
                                                          padding=0)

In [48]:
net

Network(
  (model): VGG16(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (17)

In [49]:
save_path = "/home/lincolnzjx/Desktop/Interpretation/saved/models/051"
!mkdir /home/lincolnzjx/Desktop/Interpretation/saved/models/051
torch.save(net.state_dict(), os.path.join(save_path, "0"))

In [50]:
net.load_state_dict(torch.load(os.path.join(save_path, "0")))
# print bias values. 
for name, param in net.named_parameters():
    if "bias" in name:
        print(param)
print(net)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 